In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
# import PhoPositionalData
# import PhoGui

# sys.path
# sys.cwd
# Path.cwd()
# Path.cd('PhoPy3DPositionAnalysis2021')
# import PhoPy3DPositionAnalysis2021.PhoPositionalData as phoPD

from PhoPositionalData.analysis.interactive_placeCell_config import *


# from phoPD.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from PhoPositionalData.process_data import *
# from PhoPositionalData.plot_data import *
# from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
# from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
# from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters, NamedEpoch, SessionConfig  # VideoOutputModeConfig, InteractivePlaceCellConfigs
# from PhoPositionalData.analysis.interactive_placeCell_config import get_subsession_for_epoch, print_subsession_neuron_differences

# from PhoPositionalData.load_exported import *
# # from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from PhoPositionalData.process_data import *
# from PhoPositionalData.plot_data import *
# from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
# from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
# from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters, NamedEpoch, SessionConfig  # VideoOutputModeConfig, InteractivePlaceCellConfigs
# from PhoPositionalData.analysis.interactive_placeCell_config import get_subsession_for_epoch, print_subsession_neuron_differences

In [13]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
from neuropy.core.dataSession import SessionFolderSpec, DataSessionLoader, DataSession, processDataSession

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m

# From pre-computed .mat files:
## 07: 
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
# spike_file = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikeII.mat'
# neuroscope_xml_file = Path(basedir).joinpath('2006-6-07_11-26-53.xml')

# ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'
# spike_file = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.spikeII.mat' # '2006-6-08_14-26-15.spikeII.mat' # Contains 'spike' flat structure
# neuroscope_xml_file = Path(basedir).joinpath('2006-6-08_14-26-15.xml')

# sess = core.processDataSession(basedir)
session_name = Path(basedir).parts[-1]

## KDiba Old Format:

In [21]:
# KDiba Old Format:

def kdiba_old_format_session(basedir):
    def kdiba_old_format_get_session_name(basedir):
        return Path(basedir).parts[-1]
    session_name = kdiba_old_format_get_session_name(basedir) # session_name = '2006-6-07_11-26-53'
    session_spec = SessionFolderSpec(required=[fname.format(session_name) for fname in ['{}.xml','{}.spikeII.mat','{}.position_info.mat','{}.epochs_info.mat']])
    # session_spec = SessionFolderSpec(required=['{}.xml'.format(session_name),
    #                             '{}.spikeII.mat'.format(session_name), 
    #                             '{}.position_info.mat'.format(session_name),
    #                             '{}.epochs_info.mat'.format(session_name), 
    # ])
    active_sess_config = SessionConfig(basedir, session_name=session_name, session_spec=session_spec)
    
active_sess_config = kdiba_old_format_session(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')
active_sess_config.is_resolved

True

## Bapun Format:

In [19]:
## Bapun Format:
# basedir = '/media/share/data/Bapun/Day5TwoNovel' # Linux
basedir = Path('R:\data\Bapun\Day5TwoNovel') # Windows
# basedir = '/Volumes/iNeo/Data/Bapun/Day5TwoNovel' # MacOS

def bapun_data_get_session_name(basedir):
    # Find the only .xml file to obtain the session name
    xml_files = sorted(basedir.glob("*.xml"))        
    assert len(xml_files) == 1, "Found more than one .xml file"
    file_prefix = xml_files[0].with_suffix("") # gets the session name (basically) without the .xml extension. (R:\data\Bapun\Day5TwoNovel\RatS-Day5TwoNovel-2020-12-04_07-55-09)   
    file_basename = xml_files[0].stem # file_basename: (RatS-Day5TwoNovel-2020-12-04_07-55-09)
    # print('file_prefix: {}\nfile_basename: {}'.format(file_prefix, file_basename))
    return file_basename # 'RatS-Day5TwoNovel-2020-12-04_07-55-09'

    
session_name = bapun_data_get_session_name(basedir) # 'RatS-Day5TwoNovel-2020-12-04_07-55-09'
session_spec = SessionFolderSpec(required=[fname.format(session_name) for fname in ['{}.xml','{}.neurons.npy','{}.probegroup.npy','{}.position.npy','{}.paradigm.npy']])
active_sess_config = SessionConfig(basedir, session_spec=session_spec, session_name=session_name)
active_sess_config.is_resolved

True

In [ ]:
curr_args_dict = dict()
curr_args_dict['basepath'] = basedir
# curr_args_dict['session_obj'] = DataSession()
# curr_session = DataSessionLoader.default_load_bapun_npy_session_folder(curr_args_dict)
sess = DataSessionLoader.default_load_kamran_flat_spikes_mat_session_folder(curr_args_dict)
sess







## Main Spike/Placemap plotting: